In [13]:
import pandas as pd


In [14]:
movie=pd.read_csv('movies.csv')
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [15]:
import re ##this will help us to clean the columns with empty spaces,and uneccessary links from the column

In [16]:
import re
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)
movie['clean_title']=movie['title'].apply(clean_title)

In [17]:
movie

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [18]:
movie.isnull.sum

AttributeError: 'function' object has no attribute 'sum'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  ##IT WILL TURN TITLE INTO A SET OF NUMBERS SO THAT COMPUTER CAN SEARCH IT AND GET THE ONE WE PREFER

In [ ]:

vectorizer=TfidfVectorizer(ngram_range=(1,2))
tfidf=vectorizer.fit_transform(movie["clean_title"])
   


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
def search(title):
  title=clean_title(title)
  query_vec=vectorizer.transform([title])
  similarities=cosine_similarity(query_vec, tfidf).flatten()  ##it returns how similar our tiles are related to each other
  indices=np.argpartition(similarities, -5)[-5:]
  results=movie.iloc[indices][::-1]##assuming we want to look for the most similar movies we will have to run this code
  return results

## Here we have to build an interactive search box function 

In [ ]:
import ipywidgets as widgets  ##lets do the installation of ipywidgets
from IPython.display import display

In [ ]:
movieS_input = widgets.Text(
    value = "Toy Story",
    description = "Movie Title:",
    disabled=False
)
movieS_list = widgets.Output()

def on_type(data):
    with movieS_list:
        movieS_list.clear_output()
        title=data["new"]
        if len(title) > 5:
            display(search(title))
movieS_input.observe(on_type, names='value')
display(movieS_input,movieS_list)
movieS_input

Text(value='Toy Story', description='Movie Title:')

Output()

Text(value='Toy Story', description='Movie Title:')

READ IN RATING DATASET TO RECOMMEND MOVIE WHEN USER SEARCH

In [ ]:
ratings = pd.read_csv("ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [ ]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

## We find users who like the same movie as us

In [ ]:
movie_id = 1

In [ ]:

similar_users = ratings[(ratings['movieId']==movie_id)&(ratings['rating']>=5)]['userId'].unique()
similar_users

array([    36,     75,     86, ..., 162518, 162519, 162530], dtype=int64)

## lets find the other movies they like as us

In [ ]:

similar_user_recs = ratings[(ratings['userId'].isin(similar_users))&(ratings['rating']>4)]
similar_user_recs


,userId,movieId,rating,timestamp
5101,36,1,5.0,857131378
5105,36,34,5.0,834413787
5111,36,110,5.0,834412999
5114,36,150,5.0,839928587
5127,36,260,5.0,857131062
...,...,...,...,...
24998388,162530,3706,5.0,989809041
24998389,162530,3735,5.0,989808150
24998391,162530,3763,5.0,989809659
24998392,162530,4187,5.0,989809274


# Lets find the movies graded 10% of the movies similar to that which we like


find only the movies greater than 10% liked

In [ ]:
similar=similar_user_recs.value_counts()
similar


userId  movieId  rating  timestamp 
36      1        5.0     857131378     1
107436  2355     4.5     1510504770    1
107435  94959    5.0     1439581182    1
        109374   4.5     1439581180    1
        134393   4.5     1439582224    1
                                      ..
53971   7153     4.5     1269811150    1
        7346     4.5     1269660090    1
        8665     4.5     1270439664    1
        30707    5.0     1269660409    1
162530  4321     5.0     989807602     1
Name: count, Length: 912084, dtype: int64

In [ ]:
length=len(similar_users)
length

13506

In [ ]:
similar_user_recs=similar/length
similar_user_recs

userId  movieId  rating  timestamp 
36      1        5.0     857131378     0.000074
107436  2355     4.5     1510504770    0.000074
107435  94959    5.0     1439581182    0.000074
        109374   4.5     1439581180    0.000074
        134393   4.5     1439582224    0.000074
                                         ...   
53971   7153     4.5     1269811150    0.000074
        7346     4.5     1269660090    0.000074
        8665     4.5     1270439664    0.000074
        30707    5.0     1269660409    0.000074
162530  4321     5.0     989807602     0.000074
Name: count, Length: 912084, dtype: float64

In [ ]:
similar_user_recs=similar_user_recs[similar_user_recs>.1]
similar_user_recs

Series([], Name: count, dtype: float64)

## we want to find how much people in dataset that likes avengers and movies related to avengers

In [ ]:
all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index))&(ratings['rating']>4)]
all_users_recs=all_users['movieId'].value_counts()/len(all_users['movieId'].unique())
all_users_recs


NameError: name 'ratings' is not defined

In [ ]:

rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)
rec_percentages.columns=['similar','all']
rec_percentages

NameError: name 'similar_user_recs' is not defined

In [ ]:
rec_percentages['score']=rec_percentages['similar']/rec_percentages['all']
rec_percentages=rec_percentages.sort_values("score",ascending=False)
rec_percentages

NameError: name 'rec_percentages' is not defined

In [ ]:
rec_percentages.head(10).merge(movie,left_index=True,right_on='movieId',how='left')

NameError: name 'rec_percentages' is not defined

RECOMMENDATION FUNCTION

In [ ]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings['movieId']==movie_id)&(ratings['rating']>=5)]['userId'].unique()
    similar_user_recs = ratings[(ratings['userId'].isin(similar_users))&(ratings['rating']>4)]
    all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index))&(ratings['rating']>4)]
    all_users_recs=all_users['movieId'].value_counts()/len(all_users['movieId'].unique())
    rec_percentages=pd.concat([similar_user_recs,all_users_recs],ignore_index=True)
movie_name_input=widgets.Text(
    value="Toy Story",
    description="Movie Title:",
    disabled=False
)  
recommendation_list= widgets.Output()
def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"] 
        if len(title) > 5:
            results=search(title)
            movie_id=results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))
movie_name_input.observe(on_type, names="value")  
display(movie_name_input, recommendation_list)          

Text(value='Toy Story', description='Movie Title:')

Output()